# Prepare end result data

In [ ]:
import pandas as pd
import os

# Path to actual script
script_dir = os.getcwd()

# Relative path to directory withing repository, based on attack kind
relative_path = "results/Attack_Kind_1/"

# Create full path to directory
path = os.path.join(script_dir, relative_path)
filtered_df = pd.DataFrame()
share_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Loop over all different files, collect & prepare data, store them into new .csv file
for j in range(11):
    pre = f"0_{j}"
    if j == 10: pre = "1_0"

    for k in range(11):
        imu = f"0_{k}"
        if k == 10: imu = "1_0"
        # Datei laden
        file_path = f"{path}end_result_atk1_pre{pre}_imu{imu}.csv"
        df = pd.read_csv(file_path)

        for l in share_values:
            for m in share_values:
                for n in share_values:
                    filtered_row = df[
                        (df["prob_share_disinfo"] == l) &
                        (df["prob_share_indifferent"] == m) &
                        (df["prob_share_facts"] == n)
                    ]

                    filtered_df = pd.concat([filtered_df, filtered_row[['prob_prebunk', 'prob_immune', 'prob_share_disinfo', 'prob_share_indifferent', 'prob_share_facts', 's', 'i', 'r']]])

filtered_df['mean_infected_prop'] = filtered_df['i'] / 100
filtered_df.to_csv('filtered_df.csv', index=False)


# Interactive plots with sliders

In [ ]:
import altair as alt
import pandas as pd

# Plot infected agents
alt.data_transformers.disable_max_rows()
df = pd.read_csv("filtered_df.csv")

df_cleaned = df.drop(columns=['mean_infected_prop', 's', 'r'])

# Initialize sliders
slider_prob_share_facts = alt.binding_range(min=0, max=1, step=0.1, name='prob_share_facts')
slider_prob_share_disinformation = alt.binding_range(min=0, max=1, step=0.1, name='prob_share_disinformation')
slider_prob_share_indifferent = alt.binding_range(min=0, max=1, step=0.1, name='prob_share_indifferent')

# Assign filters to sliders
slider1 = alt.selection_point(bind=slider_prob_share_facts, fields=['prob_share_facts'], name="ProbPrebunkingContent")
slider2 = alt.selection_point(bind=slider_prob_share_disinformation, fields=['prob_share_disinfo'], name="ProbDisinformation")
slider3 = alt.selection_point(bind=slider_prob_share_indifferent, fields=['prob_share_indifferent'], name="ProbIndifferent")

# Create base chart
base = alt.Chart(df_cleaned).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 90])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds'))
)

# Add filters to chart
filter_TRA = base.add_params(
    slider1, slider2, slider3
).transform_filter(
    slider1
).transform_filter(
    slider2
).transform_filter(
    slider3
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Plot for single Attack Scenario',
        ))

filter_TRA


In [ ]:
import altair as alt
import pandas as pd

# Plot resistant agents
alt.data_transformers.disable_max_rows()
df = pd.read_csv("all_results_atk1.csv")
df_cleaned = df.drop(columns=['mean_infected_prop', 's', 'i'])

# Initialize sliders
slider_prob_share_facts = alt.binding_range(min=0, max=1, step=0.1, name='prob_share_facts')
slider_prob_share_disinformation = alt.binding_range(min=0, max=1, step=0.1, name='prob_share_disinformation')
slider_prob_share_indifferent = alt.binding_range(min=0, max=1, step=0.1, name='prob_share_indifferent')

# Assign filters to sliders
slider1 = alt.selection_point(bind=slider_prob_share_facts, fields=['prob_share_facts'], name="ProbPrebunkingConten")
slider2 = alt.selection_point(bind=slider_prob_share_disinformation, fields=['prob_share_disinfo'], name="ProbDisinformation")
slider3 = alt.selection_point(bind=slider_prob_share_indifferent, fields=['prob_share_indifferent'], name="ProbIndifferent")

# Create base chart
base = alt.Chart(df_cleaned).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)', scale=alt.Scale(domain=[0, 1])),
                y=alt.Y('r', title='mean resistant (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='greens'))
)

# Add filters to chart
filter_TRA = base.add_params(
    slider1, slider2, slider3
).transform_filter(
    slider1
).transform_filter(
    slider2
).transform_filter(
    slider3
).properties(width=600, height=600,
        title=alt.TitleParams(
            text=f'Plot for single Attack Scenario',
        ))

filter_TRA


# Plot network initialization

In [ ]:
import runModel
import plotResults
import networkx as nx
from matplotlib import pyplot as plt, gridspec as gridspec
from collections import Counter

# Load example population
population = runModel.create_population(False, 100, 0.5, 0.5, 0.5, 5, 5, 0.75)
node_list, tie_list = plotResults.get_network(population)

network = nx.DiGraph()
network.add_nodes_from(node_list)
network.add_edges_from(tie_list)

node_mult = 40
degrees = [val * node_mult for (node, val) in network.degree()]
in_degrees_numeric = {node: deg for node, deg in network.in_degree()}
in_degrees_labels = {node: str(deg) for node, deg in in_degrees_numeric.items()}
lay = nx.spring_layout(network)

fig = plt.figure(figsize=(10, 15))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 0.5], width_ratios=[1])

# Plot network
net_start = fig.add_subplot(gs[0])
nx.draw_networkx_nodes(network, pos=lay, ax=net_start, node_size=degrees, node_color='blue', alpha=.75)
nx.draw_networkx_edges(network, pos=lay, arrows=True, ax=net_start, node_size=degrees, alpha=.3)

nx.draw_networkx_labels(network, pos=lay, labels=in_degrees_labels, font_size=10, font_color='white')

indegree_counts = Counter(in_degrees_numeric.values())
x_indegrees, y_counts = zip(*sorted(indegree_counts.items()))

# Plot distribution of indegree's
ax_indegree_dist = fig.add_subplot(gs[1])
ax_indegree_dist.bar(x_indegrees, y_counts, alpha=0.7, color='blue')
ax_indegree_dist.set_xlabel("indegree")
ax_indegree_dist.set_ylabel("n(agents)")
ax_indegree_dist.set_xticks(range(0, max(x_indegrees) + 5, 5))
ax_indegree_dist.set_yticks(range(0, max(y_counts) + 5, 5))

net_start.set_facecolor('none')
ax_indegree_dist.set_facecolor('none')
fig.patch.set_facecolor('white')
ax_indegree_dist.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, alpha=0.7)
plt.show()


# Plots for different attack kinds

In [1]:
import pandas as pd
import altair as alt
import os

# Path to actual script
script_dir = os.getcwd()

# Plotting infected agents
filtered_df = pd.DataFrame()
for i in range(3):
    relative_path = f"results/Attack_Kind_{i}/"
    path = os.path.join(script_dir, relative_path)
    for j in range(11):
        pre = f"0_{j}"
        if j == 10: pre = "1_0"

        for k in range(11):
            imu = f"0_{k}"
            if k == 10: imu = "1_0"
            # File loading
            file_path = f"{path}end_result_atk{i}_pre{pre}_imu{imu}.csv"
            df = pd.read_csv(file_path)

            filtered_row = df[
                (df["prob_share_disinfo"] == 0.5) &
                (df["prob_share_indifferent"] == 0.5) &
                (df["prob_share_facts"] == 0.5)
            ]

            filtered_df = pd.concat([filtered_df, filtered_row[['prob_prebunk', 'prob_immune', 'attack_kind', 's', 'i', 'r']]])

filtered_df['mean_infected_prop'] = filtered_df['i'] / 100


# Plot filtered Data
a = alt.Chart(filtered_df[filtered_df.attack_kind==0]).mark_line().encode(
    x=alt.X('prob_prebunk', title='P(r)'),
    y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 90])),
    color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds')),
).properties(title=alt.TitleParams(
        text='Attack Scenario I',
        subtitle='P(O₀) = 0.5, P(O₁) = 0.5, P(O₂) = 0.5',
        subtitleFontStyle='italic' ))

b = alt.Chart(filtered_df[filtered_df.attack_kind==1]).mark_line().encode(
    x=alt.X('prob_prebunk', title='P(r)'),
    y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 90])),
    color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds')),
).properties(title=alt.TitleParams(
        text='Attack Scenario II',
        subtitle='P(O₀) = 0.5, P(O₁) = 0.5, P(O₂) = 0.5',
        subtitleFontStyle='italic'
))

c = alt.Chart(filtered_df[filtered_df.attack_kind==2]).mark_line().encode(
    x=alt.X('prob_prebunk', title='P(r)'),
    y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 90])),
    color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds')),
).properties(title=alt.TitleParams(
        text='Attack Scenario III',
        subtitle='P(O₀) = 0.5, P(O₁) = 0.5, P(O₂) = 0.5',
        subtitleFontStyle='italic'
))

# Add empty row for centering the lower row
empty = alt.Chart(pd.DataFrame({'x': []})).mark_point().properties(width=120, height=200)

(a | b) & (empty | c | empty)

alt.VConcatChart(...)

In [2]:
import pandas as pd
import altair as alt
import numpy as np
import re
import os

# Path to actual script
script_dir = os.getcwd()

# Plotting opinion shares for different attack kinds
filtered_df = pd.DataFrame()
for i in range(3):
    relative_path = f"results/Attack_Kind_{i}/"
    path = os.path.join(script_dir, relative_path)
    file_path = f"{path}end_result_atk{i}_pre0_3_imu0_1.csv"
    df = pd.read_csv(file_path)

    filtered_row = df[
        (df["prob_share_disinfo"] == 0.5) &
        (df["prob_share_indifferent"] == 0.5) &
        (df["prob_share_facts"] == 0.5)
    ]

    filtered_df = pd.concat([filtered_df, filtered_row])

x_values = list(range(1, 101))
test = pd.DataFrame(filtered_df[filtered_df.attack_kind==1])
test2 = filtered_df.loc[filtered_df['attack_kind'] == 0, 's_shares']

raw_data0 = np.concatenate([
    filtered_df.loc[filtered_df['attack_kind'] == 0, 's_shares'].astype(str).values,
    filtered_df.loc[filtered_df['attack_kind'] == 0, 'i_shares'].astype(str).values,
    filtered_df.loc[filtered_df['attack_kind'] == 0, 'r_shares'].astype(str).values
])

def clean_and_convert(s):
    s = re.sub(r'[\[\]\n]', ' ', s)
    return np.array([float(x) for x in s.split()])

list_data0 = [clean_and_convert(item) for item in raw_data0]
flat_list0 = np.concatenate(list_data0)

df_0 = pd.DataFrame({
    'x': x_values * 3,
    'y': flat_list0,
    'line': np.repeat(['no opinion (O₀)', 'misinformation (O₁)', 'prebunking (O₂)'], 100),
})

raw_data1 = np.concatenate([
    filtered_df.loc[filtered_df['attack_kind'] == 1, 's_shares'].astype(str).values,
    filtered_df.loc[filtered_df['attack_kind'] == 1, 'i_shares'].astype(str).values,
    filtered_df.loc[filtered_df['attack_kind'] == 1, 'r_shares'].astype(str).values
])

list_data1 = [clean_and_convert(item) for item in raw_data1]

flat_list1 = np.concatenate(list_data1)

df_1 = pd.DataFrame({
    'x': x_values * 3,
    'y': flat_list1,
    'line': np.repeat(['no opinion (O₀)', 'misinformation (O₁)', 'prebunking (O₂)'], 100),
})

raw_data2 = np.concatenate([
    filtered_df.loc[filtered_df['attack_kind'] == 2, 's_shares'].astype(str).values,
    filtered_df.loc[filtered_df['attack_kind'] == 2, 'i_shares'].astype(str).values,
    filtered_df.loc[filtered_df['attack_kind'] == 2, 'r_shares'].astype(str).values
])

list_data2 = [clean_and_convert(item) for item in raw_data2]

flat_list2 = np.concatenate(list_data2)

df_2 = pd.DataFrame({
    'x': x_values * 3,
    'y': flat_list2,
    'line': np.repeat(['no opinion (O₀)', 'misinformation (O₁)', 'prebunking (O₂)'], 100),
})

color_scale = alt.Scale(
    domain=['no opinion (O₀)', 'misinformation (O₁)', 'prebunking (O₂)'],
    range=['grey', 'red', 'green']
)

a = alt.Chart(df_0).mark_line().encode(
    x=alt.X('x', title='Simulation Steps'),
    y=alt.Y('y', title='Opinion Shares'),
    color=alt.Color('line', title='Opinion Type', scale=color_scale)
).properties(width=400, height=150,
        title=alt.TitleParams(
        text='Attack Scenario I',
        subtitle='P(O₀) = 0.5, P(O₁) = 0.5, P(O₂) = 0.5',
        subtitleFontStyle='italic'
))

b = alt.Chart(df_1).mark_line().encode(
    x=alt.X('x', title='Simulation Steps'),
    y=alt.Y('y', title='Opinion Shares'),
    color=alt.Color('line', title='Opinion Type', scale=color_scale)
).properties(width=400, height=150,
        title=alt.TitleParams(
        text='Attack Scenario II',
        subtitle='P(O₀) = 0.5, P(O₁) = 0.5, P(O₂) = 0.5',
        subtitleFontStyle='italic'
))

c = alt.Chart(df_2).mark_line().encode(
    x=alt.X('x', title='Simulation Steps'),
    y=alt.Y('y', title='Opinion Shares'),
    color=alt.Color('line', title='Opinion Type', scale=color_scale)
).properties(width=400, height=150,
        title=alt.TitleParams(
        text='Attack Scenario III',
        subtitle='P(O₀) = 0.5, P(O₁) = 0.5, P(O₂) = 0.5',
        subtitleFontStyle='italic'
))

empty = alt.Chart(pd.DataFrame({'x': []})).mark_point().properties(width=170, height=150)

a & b & c

alt.VConcatChart(...)

# Influence different information volume

In [3]:
import pandas as pd
import altair as alt
import numpy as np
import re
import os

# Path to actual script
script_dir = os.getcwd()

# Relative path to directory withing repository, based on attack kind
relative_path = "results/Attack_Kind_1/"
path = os.path.join(script_dir, relative_path)

def clean_and_convert(s):
    s = re.sub(r'[\[\]\n]', ' ', s)
    return np.array([float(x) for x in s.split()])

charts = []

# Loop over all combinations of prob_share_disinfo, prob_share_indifferent, prob_share_facts
share_values_full = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
share_values = [0.1, 0.4, 0.9]
path_values = ['0_0', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '1_0']
pr_pos = 3
pv_pos = 1

pr = share_values_full[pr_pos]
pv = share_values_full[pv_pos]
pr_path = path_values[pr_pos]
pv_path = path_values[pv_pos]

file_path = f"{path}end_result_atk1_pre{pr_path}_imu{pv_path}.csv"
df = pd.read_csv(file_path)

# Filter only for one attack_kind
df_attack_kind_1 = df[df['attack_kind'] == 1]

for l in share_values:

    filtered_df = df_attack_kind_1[
        (df_attack_kind_1["prob_share_disinfo"] == l) &
        (df_attack_kind_1["prob_share_indifferent"] == l) &
        (df_attack_kind_1["prob_share_facts"] == l)
    ]

    x_values = list(range(1, 101))

    raw_data0 = np.concatenate([
        filtered_df.loc[filtered_df['attack_kind'] == 1, 's_shares'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'i_shares'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'r_shares'].astype(str).values
    ])

    list_data0 = [clean_and_convert(item) for item in raw_data0]
    flat_list0 = np.concatenate(list_data0) if list_data0 else []

    df_0 = pd.DataFrame({
        'x': x_values * 3,
        'y': flat_list0,
        'line': np.repeat(['no opinion (O₀)', 'misinformation (O₁)', 'prebunking (O₂)'], 100),
    })

    color_scale = alt.Scale(
        domain=['no opinion (O₀)', 'misinformation (O₁)', 'prebunking (O₂)'],
        range=['grey', 'red', 'green']
    )

    a = alt.Chart(df_0).mark_line().encode(
        x=alt.X('x', title='Simulation Steps'),
        y=alt.Y('y', title='Opinion Shares', scale=alt.Scale(domain=[0, 100])),
        color=alt.Color('line', title='Opinion Type', scale=color_scale)
    ).properties(width=400, height=150,
        title=alt.TitleParams(
            text=f'Attack Scenario II, P(r) = {pr}, P(v) = {pv}',
            subtitle=f'P(O₀) = {l}, P(O₁) = {l}, P(O₂) = {l}',
            subtitleFontStyle='italic'
        ))

    charts.append(a)

rows = []
for i in range(0, len(charts), 1):
    rows.append(alt.hconcat(*charts[i:i+1]))

final_chart = alt.vconcat(*rows)

final_chart


alt.VConcatChart(...)

In [4]:
import pandas as pd
import altair as alt
import numpy as np

# Path to actual script
script_dir = os.getcwd()

# Relative path to directory withing repository, based on attack kind
relative_path = "results/Attack_Kind_1/"
path = os.path.join(script_dir, relative_path)

def clean_and_convert(s):
    s = re.sub(r'[\[\]\n]', ' ', s)
    return np.array([float(x) for x in s.split()])

charts = []

# Loop over all combinations of prob_share_disinfo, prob_share_indifferent, prob_share_facts
share_values_full = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
share_values = [0.1, 0.4, 0.9]
path_values = ['0_0', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '1_0']
pr_pos = 3
pv_pos = 1

pr = share_values_full[pr_pos]
pv = share_values_full[pv_pos]
pr_path = path_values[pr_pos]
pv_path = path_values[pv_pos]

file_path = f"{path}end_result_atk1_pre{pr_path}_imu{pv_path}.csv"
df = pd.read_csv(file_path)

df_attack_kind_1 = df[df['attack_kind'] == 1]

for l in share_values:

    filtered_df = df_attack_kind_1[
        (df_attack_kind_1["prob_share_disinfo"] == l) &
        (df_attack_kind_1["prob_share_indifferent"] == l) &
        (df_attack_kind_1["prob_share_facts"] == l)
    ]

    x_values = list(range(1, 101))

    raw_data0 = np.concatenate([
        filtered_df.loc[filtered_df['attack_kind'] == 1, 's_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'i_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'r_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'ar_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'ui_status'].astype(str).values
    ])

    list_data0 = [clean_and_convert(item) for item in raw_data0]
    flat_list0 = np.concatenate(list_data0) if list_data0 else []

    df_0 = pd.DataFrame({
        'x': x_values * 5,
        'y': flat_list0,
        'line': np.repeat(['susceptible', 'infected', 'resistant', 'prebunking agent', 'dark agent'], 100),
    })

    color_scale = alt.Scale(
        domain=['susceptible', 'infected', 'resistant', 'prebunking agent', 'dark agent'],
        range=['grey', 'pink', 'lightgreen', 'darkgreen', 'red']
    )

    a = alt.Chart(df_0).mark_line().encode(
        x=alt.X('x', title='Simulation Steps'),
        y=alt.Y('y', title='# Status', scale=alt.Scale(domain=[0, 100])),
        color=alt.Color('line', title='Status', scale=color_scale)
    ).properties(width=400, height=150,
        title=alt.TitleParams(
            text=f'Attack Scenario II, P(r) = {pr}, P(v) = {pv}',
            subtitle=f'P(O₀) = {l}, P(O₁) = {l}, P(O₂) = {l}',
            subtitleFontStyle='italic'
        ))

    charts.append(a)

rows = []
for i in range(0, len(charts), 1):
    rows.append(alt.hconcat(*charts[i:i+1]))

final_chart = alt.vconcat(*rows)

final_chart


alt.VConcatChart(...)

# Influence indifferent propagation speed (opinion 1, P(O₁))

In [5]:
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

# Read .csv file with prepared simulation results
df = pd.read_csv("filtered_df.csv")

dfa = df[
        (df["prob_share_disinfo"] == 0.5) &
        (df["prob_share_indifferent"] == 0.3) &
        (df["prob_share_facts"] == 0.5)
    ]

dfb = df[
        (df["prob_share_disinfo"] == 0.5) &
        (df["prob_share_indifferent"] == 0.9) &
        (df["prob_share_facts"] == 0.5)
    ]

# Create base chart
a = alt.Chart(dfa).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds'))
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Attack Scenario II',
            subtitle=f'P(O₀) = {0.3}, P(O₁) = {0.5}, P(O₂) = {0.5}',
            subtitleFontStyle='italic'
        ))

b = alt.Chart(dfb).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds'))
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Attack Scenario II',
            subtitle=f'P(O₀) = {0.9}, P(O₁) = {0.5}, P(O₂) = {0.5}',
            subtitleFontStyle='italic'
        ))

a | b


alt.HConcatChart(...)

# Influence misinformation propagation speed (opinion 1, P(O₁))

In [6]:
import pandas as pd
import altair as alt
import numpy as np
import re

# Relative path to directory withing repository, based on attack kind
relative_path = "results/Attack_Kind_1/"
path = os.path.join(script_dir, relative_path)

def clean_and_convert(s):
    s = re.sub(r'[\[\]\n]', ' ', s)
    return np.array([float(x) for x in s.split()])

charts = []

# Loop over all combinations of prob_share_disinfo, prob_share_indifferent, prob_share_facts
share_values_full = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
share_values = [0.3, 0.9]
path_values = ['0_0', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '1_0']
pr_pos = 1
pv_pos = 1

pr = share_values_full[pr_pos]
pv = share_values_full[pv_pos]
pr_path = path_values[pr_pos]
pv_path = path_values[pv_pos]

file_path = f"{path}end_result_atk1_pre{pr_path}_imu{pv_path}.csv"
df = pd.read_csv(file_path)

df_attack_kind_1 = df[df['attack_kind'] == 1]

for l in share_values:

    filtered_df = df_attack_kind_1[
        (df_attack_kind_1["prob_share_disinfo"] == l) &
        (df_attack_kind_1["prob_share_indifferent"] == 0.5) &
        (df_attack_kind_1["prob_share_facts"] == 0.5)
    ]

    x_values = list(range(1, 101))

    raw_data0 = np.concatenate([
        filtered_df.loc[filtered_df['attack_kind'] == 1, 's_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'i_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'r_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'ar_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'ui_status'].astype(str).values
    ])

    list_data0 = [clean_and_convert(item) for item in raw_data0]
    flat_list0 = np.concatenate(list_data0) if list_data0 else []

    df_0 = pd.DataFrame({
        'x': x_values * 5,
        'y': flat_list0,
        'line': np.repeat(['susceptible', 'infected', 'resistant', 'prebunking agent', 'dark agent'], 100),
    })

    color_scale = alt.Scale(
        domain=['susceptible', 'infected', 'resistant', 'prebunking agent', 'dark agent'],
        range=['grey', 'pink', 'lightgreen', 'darkgreen', 'red']
    )

    a = alt.Chart(df_0).mark_line().encode(
        x=alt.X('x', title='Simulation Steps'),
        y=alt.Y('y', title='# Status', scale=alt.Scale(domain=[0, 100])),
        color=alt.Color('line', title='Status', scale=color_scale)
    ).properties(width=400, height=150,
        title=alt.TitleParams(
            text=f'Attack Scenario II, P(r) = {pr}, P(v) = {pv}',
            subtitle=f'P(O₀) = {0.5}, P(O₁) = {l}, P(O₂) = {0.5}',
            subtitleFontStyle='italic'
        ))

    charts.append(a)

rows = []
for i in range(0, len(charts), 1):
    rows.append(alt.hconcat(*charts[i:i+1]))

final_chart = alt.vconcat(*rows)

final_chart


alt.VConcatChart(...)

In [7]:
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

# Read .csv file with prepared simulation results
df = pd.read_csv("filtered_df.csv")

dfa = df[
        (df["prob_share_disinfo"] == 0.3) &
        (df["prob_share_indifferent"] == 0.5) &
        (df["prob_share_facts"] == 0.5)
    ]

dfb = df[
        (df["prob_share_disinfo"] == 0.9) &
        (df["prob_share_indifferent"] == 0.5) &
        (df["prob_share_facts"] == 0.5)
    ]

# Create base chart
a = alt.Chart(dfa).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds'))
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Attack Scenario II',
            subtitle=f'P(O₀) = {0.5}, P(O₁) = {0.3}, P(O₂) = {0.5}',
            subtitleFontStyle='italic'
        ))

b = alt.Chart(dfb).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('i', title='mean infected (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='reds'))
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Attack Scenario II',
            subtitle=f'P(O₀) = {0.5}, P(O₁) = {0.9}, P(O₂) = {0.5}',
            subtitleFontStyle='italic'
        ))

a | b


alt.HConcatChart(...)

# Influence prebunking content propagation speed (opinion 2, P(O₂))

In [8]:
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

# Read .csv file with prepared simulation results
df = pd.read_csv("filtered_df.csv")

dfa = df[
        (df["prob_share_disinfo"] == 0.5) &
        (df["prob_share_indifferent"] == 0.5) &
        (df["prob_share_facts"] == 0.3)
    ]

dfb = df[
        (df["prob_share_disinfo"] == 0.5) &
        (df["prob_share_indifferent"] == 0.5) &
        (df["prob_share_facts"] == 0.9)
    ]

# Create base chart
a = alt.Chart(dfa).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('r', title='mean resistant (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='greens'))
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Attack Scenario II',
            subtitle=f'P(O₀) = {0.5}, P(O₁) = {0.5}, P(O₂) = {0.3}',
            subtitleFontStyle='italic'
        ))

b = alt.Chart(dfb).mark_line().encode(
                x=alt.X('prob_prebunk', title='P(r)'),
                y=alt.Y('r', title='mean resistant (%)', scale=alt.Scale(domain=[0, 100])),
                color=alt.Color('prob_immune', title='P(v)', scale=alt.Scale(scheme='greens'))
).properties(width=400, height=400,
        title=alt.TitleParams(
            text=f'Attack Scenario II',
            subtitle=f'P(O₀) = {0.5}, P(O₁) = {0.5}, P(O₂) = {0.9}',
            subtitleFontStyle='italic'
        ))
a | b

alt.HConcatChart(...)

In [9]:
import pandas as pd
import altair as alt
import numpy as np
import re

# Relative path to directory withing repository, based on attack kind
relative_path = "results/Attack_Kind_1/"
path = os.path.join(script_dir, relative_path)

def clean_and_convert(s):
    s = re.sub(r'[\[\]\n]', ' ', s)
    return np.array([float(x) for x in s.split()])

charts = []

# Loop over all combinations of prob_share_disinfo, prob_share_indifferent, prob_share_facts
share_values_full = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
share_values = [0.3, 0.9, 0.3, 0.9]
path_values = ['0_0', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '1_0']
pr_pos = 1
pv_pos = 1

pr_pos2 = 1
pv_pos2 = 3

pr = share_values_full[pr_pos]
pv = share_values_full[pv_pos]
pr_path = path_values[pr_pos]
pv_path = path_values[pv_pos]

pr2 = share_values_full[pr_pos2]
pv2 = share_values_full[pv_pos2]
pr_path2 = path_values[pr_pos2]
pv_path2 = path_values[pv_pos2]


file_path = f"{path}end_result_atk1_pre{pr_path}_imu{pv_path}.csv"
df = pd.read_csv(file_path)

file_path2 = f"{path}end_result_atk1_pre{pr_path2}_imu{pv_path2}.csv"
df2 = pd.read_csv(file_path2)

df = pd.concat([df, df2], ignore_index=True)
df_attack_kind_1 = df[df['attack_kind'] == 1]
i = 0
for l in share_values:

    if i >= 2:
        pr = pr2
        pv = pv2

    filtered_df = df_attack_kind_1[
        (df_attack_kind_1["prob_share_disinfo"] == 0.5) &
        (df_attack_kind_1["prob_share_indifferent"] == 0.5) &
        (df_attack_kind_1["prob_share_facts"] == l) &
        (df_attack_kind_1["prob_immune"] == pv) &
        (df_attack_kind_1["prob_prebunk"] == pr)
    ]

    x_values = list(range(1, 101))

    raw_data0 = np.concatenate([
        filtered_df.loc[filtered_df['attack_kind'] == 1, 's_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'i_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'r_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'ar_status'].astype(str).values,
        filtered_df.loc[filtered_df['attack_kind'] == 1, 'ui_status'].astype(str).values
    ])

    list_data0 = [clean_and_convert(item) for item in raw_data0]
    flat_list0 = np.concatenate(list_data0) if list_data0 else []

    df_0 = pd.DataFrame({
        'x': x_values * 5,
        'y': flat_list0,
        'line': np.repeat(['susceptible', 'infected', 'resistant', 'prebunking agent', 'dark agent'], 100),
    })

    color_scale = alt.Scale(
        domain=['susceptible', 'infected', 'resistant', 'prebunking agent', 'dark agent'],
        range=['grey', 'pink', 'lightgreen', 'darkgreen', 'red']
    )

    a = alt.Chart(df_0).mark_line().encode(
        x=alt.X('x', title='Simulation Steps'),
        y=alt.Y('y', title='# Status', scale=alt.Scale(domain=[0, 100])),
        color=alt.Color('line', title='Status', scale=color_scale)
    ).properties(width=400, height=150,
        title=alt.TitleParams(
            text=f'Attack Scenario II, P(r) = {pr}, P(v) = {pv}',
            subtitle=f'P(O₀) = {0.5}, P(O₁) = {0.5}, P(O₂) = {l}',
            subtitleFontStyle='italic'
        ))

    charts.append(a)
    i+=1

rows = []
for i in range(0, len(charts), 2):
    rows.append(alt.hconcat(*charts[i:i+2]))

final_chart = alt.vconcat(*rows)

final_chart


alt.VConcatChart(...)